# Sentiment Analysis

Sentiment analysis helps in identifying opinions, emotions, and attitudes expressed in text data, making it highly valuable for various applications, such as product reviews, social media monitoring, and customer feedback analysis.

You will learn:

* **Using pre trained sentiment analysis model**
* **Fine tune sentiment analysis models**

## Use Pre Trained Model

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


### Import the necessary libraries

In [2]:
from transformers import pipeline

### Create a sentiment analysis pipeline

We initialize a sentiment analysis pipeline using the pretrained model distilbert-base-uncased-finetuned-sst-2-english.

In [ ]:
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is 

### Provide input text

You can provide any text you want to analyze for sentiment.

In [ ]:
text = "I love the way transformers are making NLP tasks so much easier!"

### Analyze the sentiment

Pass the text into the sentiment analyzer pipeline to determine the sentiment.


In [ ]:
result = sentiment_analyzer(text)
print(result)

[{'label': 'POSITIVE', 'score': 0.9994403719902039}]


### Explanation of Output

*   label: The sentiment of the text (e.g., POSITIVE, NEGATIVE).
*   score: A confidence score indicating how certain the model is about its prediction.

## Fine Tuning

There are significant benefits to using a pretrained model. It reduces computation costs, your carbon footprint, and allows you to use state-of-the-art models without having to train one from scratch. 🤗 Transformers provides access to thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training technique. In this tutorial, you will fine-tune a pretrained model with a deep learning framework of your choice:

* Fine-tune a pretrained model with 🤗 Transformers Trainer.
* Fine-tune a pretrained model in TensorFlow with Keras.

### Install and Import Libraries

Train a TensorFlow model with Keras
You can also train 🤗 Transformers models in TensorFlow with the Keras API!

Loading data for Keras
When you want to train a 🤗 Transformers model with the Keras API, you need to convert your dataset to a format that Keras understands. If your dataset is small, you can just convert the whole thing to NumPy arrays and pass it to Keras. Let’s try that first before we do anything more complicated.

First, load a dataset. We’ll use the CoLA dataset from the GLUE benchmark, since it’s a simple binary text classification task, and just take the training split for now.

### Load the Data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset = dataset["train"]  # Just take the training split for now

### Take a Look at the Data

In [ ]:
dataset

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 8551
})

In [ ]:
print(f'First sentence: {dataset["sentence"][0]}')
print(f'First label: {dataset["label"][0]}')
print(f'First idx: {dataset["idx"][0]}')

First sentence: Our friends won't buy this analysis, let alone the next one we propose.
First label: 1
First idx: 0


### Load a Tokenizer

Next, load a tokenizer and tokenize the data as NumPy arrays. Note that the labels are already a list of 0 and 1s, so we can just convert that directly to a NumPy array without tokenization!

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
tokenized_data = tokenizer(dataset["sentence"], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(dataset["label"])  # Label is already an array of 0 and 1

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Finally, load, [compile](https://keras.io/api/models/model_training_apis/#compile-method), and fit the model. Note that Transformers models all have a default task-relevant loss function, so you don’t need to specify one unless you want to:

### Load the Model and Fine Tuning it

In [ ]:
from transformers import TFAutoModelForSequenceClassification, pipeline

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased")
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer='Adam')  # No loss argument!

model.fit(tokenized_data, labels)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


268/268 [==============================] - 119s 310ms/step - loss: 0.6487


**Note:**
You don’t have to pass a loss argument to your models when you compile() them! Hugging Face models automatically choose a loss that is appropriate for their task and model architecture if this argument is left blank. You can always override this by specifying a loss yourself if you want to!

This approach works great for smaller datasets, but for larger datasets, you might find it starts to become a problem. Why? Because the tokenized array and labels would have to be fully loaded into memory, and because NumPy doesn’t handle “jagged” arrays, so every tokenized sample would have to be padded to the length of the longest sample in the whole dataset. That’s going to make your array even bigger, and all those padding tokens will slow down training too!

### Save the Model

In [ ]:
model.save_pretrained("./cola_finetuned_model")
tokenizer.save_pretrained("./cola_finetuned_model")

('./cola_finetuned_model/tokenizer_config.json',
 './cola_finetuned_model/special_tokens_map.json',
 './cola_finetuned_model/vocab.txt',
 './cola_finetuned_model/added_tokens.json',
 './cola_finetuned_model/tokenizer.json')

### Load the model

In [ ]:
classifier = pipeline("text-classification", model="/content/cola_finetuned_model", tokenizer="/content/cola_finetuned_model", return_all_scores=False)

Some layers from the model checkpoint at /content/cola_finetuned_model were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/cola_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Hardware accelerator e.g. GPU is available in the 

### Inference

In [ ]:
sentence = "This is an acceptable sentence."
result = classifier(sentence)
print(f"Prediction: {result}")

Prediction: [{'label': 'LABEL_1', 'score': 0.7155210375785828}]
